# Explicit, suggestive and violent content detection using Amazon Rekognition

***
This notebook provides a walkthrough of [content moderation APIs](https://docs.aws.amazon.com/rekognition/latest/dg/moderation.html) in Amazon Rekognition. You can quickly identify inappropriate content in your video and image libraries.
***

# Initialize stuff

In [ ]:
# Initialise Notebook
import boto3
from IPython.display import Image as IImage, display
from IPython.display import HTML, display
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [ ]:
# Curent AWS Region. Use this to choose corresponding S3 bucket with sample content

mySession = boto3.session.Session()
awsRegion = mySession.region_name

In [ ]:
# Init clients
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
# S3 bucket that contains sample images and videos

# We are providing sample images and videos in this bucket so
# you do not have to manually download/upload test images and videos.

bucketName = "aws-workshops-" + awsRegion

# Content moderation in Images
***

In [ ]:
imageName = "content-moderation/media/weapon.png"

#### Call Rekognition to detect objects in the image

In [ ]:
# Call Amazon Rekognition to detect unsafe content in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/moderation.html

detectModerationLabelsResponse = rekognition.detect_moderation_labels(
   Image={
       'S3Object': {
           'Bucket': bucketName,
           'Name': imageName,
       }
   }
)

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageName})))

#### Review the raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Moderation API
# In the JSON response below, you will see Moderation Labels, confidence score and additional information.

display(detectModerationLabelsResponse)

#### Display list of detected moderation labels

In [ ]:
for label in detectModerationLabelsResponse["ModerationLabels"]:
    print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
    print("  - Parent: {}".format(label["ParentName"]))

# Content moderation in videos

Content Moderation in video is an async operation. 
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html
 - We first start content moderation job which returns a Job Id.
 - We can then call `get_content_moderation` to get the job status and after job is complete, we can get moderation results.
 - In production use cases, you would usually use StepFucntion or SNS topic to get notified when job is complete.
***

In [ ]:
videoName = "content-moderation/media/weapon.mov"

#### Call Rekognition to start a job for content moderation

In [ ]:
# Start content moderation job
startModerationLabelDetection = rekognition.start_content_moderation(
    Video={
        'S3Object': {
            'Bucket': bucketName,
            'Name': videoName,
        }
    },
)

moderationJobId = startModerationLabelDetection['JobId']
display("Job Id: {0}".format(moderationJobId))

### Additional (Optional) Request Attributes

ClientRequestTokenL
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html#rekognition-StartContentModeration-request-ClientRequestToken

JobTag:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html#rekognition-StartContentModeration-request-JobTag

MinConfidence:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html#rekognition-StartContentModeration-request-MinConfidence

NotificationChannel:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html#rekognition-StartContentModeration-request-NotificationChannel


#### Wait for content moderation job to complete

In [ ]:
# Wait for content moderation job to complete
# In production use cases, you would usually use StepFucntion or SNS topic to get notified when job is complete.
getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP'
)

while(getContentModeration['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP')
    
display(getContentModeration['JobStatus'])

#### Review raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Content Moderation API
# In the JSON response below, you will see list of detected content
# For each detected object, you will see information like Timestamp

display(getContentModeration)

#### Display list of potentially unsafe content in the video

In [ ]:
theObjects = {}

strDetail = "Moderation labels in video<br>=======================================<br>"
strOverall = "Moderation labels in the overall video:<br>=======================================<br>"

# Potentially unsafe detected in each frame
for obj in getContentModeration['ModerationLabels']:
    ts = obj ["Timestamp"]
    cconfidence = obj['ModerationLabel']["Confidence"]
    oname = obj['ModerationLabel']["Name"]
    strDetail = strDetail + "At {} ms: {} (Confidence: {})<br>".format(ts, oname, round(cconfidence,2))
    if oname in theObjects:
        cojb = theObjects[oname]
        theObjects[oname] = {"Name" : oname, "Count": 1+cojb["Count"]}
    else:
        theObjects[oname] = {"Name" : oname, "Count": 1}

# Unique objects detected in video
for theObject in theObjects:
    strOverall = strOverall + "Name: {}, Count: {}<br>".format(theObject, theObjects[theObject]["Count"])

# Display results
display(HTML(strOverall))
#display(HTML(strDetail))

#### Show video in the player

In [ ]:
# Show video in a player

s3VideoUrl = s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': videoName})

videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3VideoUrl)

videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)

display(HTML(videoui))

In [ ]:
listui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(strDetail)
display(HTML(listui))

***
### References
- https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectModerationLabels.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetContentModeration.html

***